# Exploration

In [11]:
import requests

BASE_URL = "https://portail-api-data.montpellier3m.fr"

def get_ecocounter_timeseries(ecocounter_id, from_date, to_date):
    url = f"{BASE_URL}/ecocounter_timeseries/{ecocounter_id}/attrs/intensity"
    params = {
        "fromDate": from_date,  # ex: "2024-01-01T00:00:00"
        "toDate": to_date       # ex: "2024-01-07T23:59:59"
    }
    r = requests.get(url, params=params)
    r.raise_for_status()
    return r.json()

def list_ecocounters():
    url = f"{BASE_URL}/ecocounter"
    r = requests.get(url)
    r.raise_for_status()
    return r.json()

counters = list_ecocounters()
print(f"Nombre d'écocompteurs : {len(counters)}")

ecocounter_id = counters[19]["id"]  # à remplacer par un ID réel issu de l'appel précédent
print(ecocounter_id)
data = get_ecocounter_timeseries(
    ecocounter_id,
    "2024-01-01T00:00:00",
    "2024-01-07T23:59:59"
)

print(data)

Nombre d'écocompteurs : 20
urn:ngsi-ld:EcoCounter:X2H21070346
{'attrName': 'intensity', 'entityId': 'urn:ngsi-ld:EcoCounter:X2H21070346', 'entityType': 'EcoCounter', 'index': ['2024-01-01T00:00:00.000+00:00', '2024-01-01T01:00:00.000+00:00', '2024-01-01T02:00:00.000+00:00', '2024-01-01T03:00:00.000+00:00', '2024-01-01T04:00:00.000+00:00', '2024-01-01T05:00:00.000+00:00', '2024-01-01T06:00:00.000+00:00', '2024-01-01T07:00:00.000+00:00', '2024-01-01T08:00:00.000+00:00', '2024-01-01T09:00:00.000+00:00', '2024-01-01T10:00:00.000+00:00', '2024-01-01T11:00:00.000+00:00', '2024-01-01T12:00:00.000+00:00', '2024-01-01T13:00:00.000+00:00', '2024-01-01T14:00:00.000+00:00', '2024-01-01T15:00:00.000+00:00', '2024-01-01T16:00:00.000+00:00', '2024-01-01T17:00:00.000+00:00', '2024-01-01T18:00:00.000+00:00', '2024-01-01T19:00:00.000+00:00', '2024-01-01T20:00:00.000+00:00', '2024-01-01T21:00:00.000+00:00', '2024-01-01T22:00:00.000+00:00', '2024-01-01T23:00:00.000+00:00', '2024-01-02T00:00:00.000+00:00',